In [1]:
from tqdm import tqdm
from collections import Counter
from itertools import combinations
from nltk.stem import PorterStemmer
import joblib 
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split

import imp
import copy
import pickle
import multiprocessing

import numpy as np
import pandas as pd
import utils as my_utils
import ELJST_script_unigram as lda
import matplotlib.pyplot as plt

In [2]:
min_df = 5
max_df = .5
maxIters = 20

beta = .01
gamma = 10
n_topics = 5
n_sentiment = 5
lambda_param = 1.0

In [3]:
dataset_name = "imdb_reviews_2_20000"

In [4]:
dataset = pd.read_pickle("datasets/"+ dataset_name + "_dataset")

In [5]:
n_sentiment = dataset.sentiment.unique().shape[0]

In [6]:
dataset.text.apply(lambda x: len(x.split(" "))).mean()

35.19305

In [7]:
embedding_name = "fasttext_0.3"

In [9]:
similar_words = pickle.load(open("resources/archive/"+ dataset_name + "_" +embedding_name + ".pickle","rb"))

In [10]:
%%time
for s in similar_words:
    for i in s.keys():
        k = list(set(s[i]))
        if i in k:
            k.remove(i)
        s[i] = k

CPU times: user 2.76 s, sys: 85.1 ms, total: 2.84 s
Wall time: 2.84 s


In [11]:
# embedding_name = "glove_0.3"

In [12]:
# similar_words = [{} for i in range(dataset.shape[0])]

In [13]:
alpha = 0.1/n_topics * np.ones(n_topics)
gamma = [gamma/(n_topics*n_sentiment)]*n_sentiment

In [14]:
imp.reload(lda)

<module 'ELJST_script_unigram' from '/home/william18026/ELJST-Paper/ELJST_script_unigram.py'>

In [15]:
sampler = lda.SentimentLDAGibbsSampler(n_topics, alpha, beta, gamma, numSentiments=n_sentiment, 
                                       SentimentRange = n_sentiment, max_df = max_df, min_df = min_df, 
                                       lambda_param = lambda_param)

In [16]:
%%time
sampler._initialize_(reviews = dataset.text.tolist(), labels = dataset.sentiment.tolist())

8833 (20000, 8833)
CPU times: user 1min 33s, sys: 3.85 s, total: 1min 37s
Wall time: 1min 37s


In [ ]:
sampler.run(name=embedding_name, reviews=dataset.text.tolist(), labels=dataset.sentiment.tolist(), 
            similar_words=similar_words, mrf=True, maxIters=20, debug=True)

  0%|          | 2/20000 [00:00<21:21, 15.60it/s]

** fasttext_0.3 0


  0%|          | 2/20000 [00:00<18:05, 18.42it/s]

** fasttext_0.3 1


  0%|          | 2/20000 [00:00<23:35, 14.13it/s]

...

  0%|          | 4/20000 [00:00<10:07, 32.93it/s]

** fasttext_0.3 3


  0%|          | 4/20000 [00:00<10:09, 32.79it/s]

** fasttext_0.3 4


 37%|███▋      | 7457/20000 [08:49<15:50, 13.20it/s]

In [ ]:
def conditionalDistribution_new(sampler, d, v, similar_words, mrf = True, debug_mode=False):
        """
        Calculates the (topic, sentiment) probability for word v in document d
        Returns:    a matrix (numTopics x numSentiments) storing the probabilities
        """
        probabilities_ts = np.ones((sampler.numTopics, sampler.numSentiments))
        topic_assignment1 = np.ones((sampler.numTopics, sampler.numSentiments))
        
        firstFactor = (sampler.n_dt[d] + sampler.alphaVec) / \
            (sampler.n_d[d] + np.sum(sampler.alphaVec))
        
        secondFactor = np.zeros((sampler.numTopics,sampler.numSentiments))
        for k in range(sampler.numTopics):
            secondFactor[k,:] = (sampler.n_dts[d, k, :] + sampler.sentimentprior[d]) / \
                (sampler.n_dt[d, k] + np.sum(sampler.sentimentprior[d]))
        thirdFactor = (sampler.n_vts[v, :, :] + sampler.beta) / \
            (sampler.n_ts + sampler.n_vts.shape[0] * sampler.beta)
        
        probabilities_ts *= firstFactor[:, np.newaxis]
        probabilities_ts *= secondFactor * thirdFactor
        #probabilities_ts = np.exp(probabilities_ts)
        probabilities_ts /= np.sum(probabilities_ts)
        
        if mrf == True and sampler.lambda_param != 0:
            all_children = np.zeros(sampler.wordOccuranceMatrix_shape).astype(int)
            try:
                all_children[similar_words[v]] = 1
            except:
                pass
            
            # all_children = similar_words[v,:] #[d][i1,:]
            new_C = sampler.vts[all_children,:, :]
            topic_assignment = new_C.sum(0)
            #topic_assignment = np.exp(topic_assignment)
            topic_assignment /= topic_assignment.sum()
            topic_assignment1 = np.exp(sampler.lambda_param * topic_assignment)
            topic_assignment1 /= topic_assignment1.sum()
            
        if debug_mode == True:
            print (probabilities_ts, topic_assignment1)
        
        probabilities_ts *= topic_assignment1
        probabilities_ts /= np.sum(probabilities_ts)

        if debug_mode == True:
            print (probabilities_ts)
            
        return probabilities_ts

In [ ]:
sampler.n_ts + sampler.n_vts.shape[0] * sampler.beta

In [ ]:
(sampler.n_vts[0,:,:] + sampler.beta)/(sampler.n_ts + sampler.n_vts.shape[0] * sampler.beta)

In [ ]:
d = 7054

In [ ]:
dataset.iloc[d].text

In [ ]:
sampler.wordOccuranceMatrix[d].sum()

In [ ]:
print (conditionalDistribution_new(sampler,d,0,similar_words,debug_mode=True))

In [ ]:
sampler.loglikelihood()

In [ ]:
joblib.dump(sampler, "dummy")

In [ ]:
s = joblib.load("dummy")

# Evaluations

In [ ]:
plt.plot(sampler.loglikelihood_history)

In [ ]:
silhouette_score(euclidean_distances(sampler.wordOccuranceMatrix),
                 sampler.dt_distribution.argmax(axis=1), metric='precomputed')

In [ ]:
davies_bouldin_score(sampler.wordOccuranceMatrix, sampler.dt_distribution.argmax(axis=1))

In [ ]:
my_utils.coherence_score(sampler.wordOccuranceMatrix, list(sampler.getTopKWords(5).values()), sampler.vocabulary)

In [ ]:
%%time
my_utils.get_hscore_multi(sampler.dt_distribution, sampler.wordOccuranceMatrix, n_topics, 500)

In [ ]:
sampler.loglikelihood()

In [ ]:
sampler.perplexity()

### Appendix

In [ ]:
# p = [item for sublist in dataset['cleaned'].tolist() for item in sublist]

In [ ]:
# sorted(Counter(p))

In [ ]:
# def process_l(s):
#     return [i.lemma_ for i in sp(s) if i.lemma_ not in '-PRON-']

In [ ]:
# l = dataset['text'].tolist()

In [ ]:
# pool = multiprocessing.Pool(n_cores)
# processed_l = pool.map(process_l, l)
# pool.close()

In [ ]:
# joblib.dump(sampler, "resources/sampler_20iter_0.5_1")

In [ ]:
# pickle_out = open("resources/amazon_muiscal_glove_0.4.pickle","wb")
# pickle.dump(similar_words, pickle_out)
# pickle_out.close()